https://www.kaggle.com/code/tyrionlannisterlzy/xgboost-dnn-ensemble-lb-0-980

In [2]:
!pip install catboost
!pip install xgboost
!pip install seaborn

     |████████████████████████████████| 76.6 MB 2.2 MB/s 


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from catboost import Pool, CatBoostClassifier, cv
from pandas.core.common import random_state
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, StratifiedShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.utils import shuffle
from xgboost import XGBClassifier

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/edu_courses/kaggle/tabular_playground_series_apr_2022/train.csv')

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/edu_courses/kaggle/tabular_playground_series_apr_2022/test.csv')

In [ ]:
df_train_lables = pd.read_csv('/content/drive/MyDrive/edu_courses/kaggle/tabular_playground_series_apr_2022/train_labels.csv')

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_train_lables.head()

In [ ]:
df_train_merge = df_train.merge(df_train_lables)

In [ ]:
df_train_merge.info()

In [ ]:
features_train = df_train_merge.drop('state', axis=1)
target_train = df_train_merge['state']

In [ ]:
features_train, features_valid = train_test_split(features_train, test_size=0.25, random_state=123)
target_train, target_valid = train_test_split(target_train, test_size=0.25, random_state=123)

In [ ]:
target_train.value_counts()

In [55]:
cbc_model = CatBoostClassifier(eval_metric='AUC:hints=skip_train~false', use_best_model=True, random_seed=123)

In [56]:
cbc_model.fit(features_train, target_train, eval_set=(features_valid, target_valid))

Learning rate set to 0.181392
0:	learn: 0.5829896	test: 0.5823466	best: 0.5823466 (0)	total: 536ms	remaining: 8m 55s
1:	learn: 0.6021046	test: 0.6022658	best: 0.6022658 (1)	total: 945ms	remaining: 7m 51s
2:	learn: 0.6154996	test: 0.6157043	best: 0.6157043 (2)	total: 1.42s	remaining: 7m 50s
3:	learn: 0.6160537	test: 0.6161306	best: 0.6161306 (3)	total: 1.85s	remaining: 7m 41s
4:	learn: 0.6179551	test: 0.6180297	best: 0.6180297 (4)	total: 2.33s	remaining: 7m 43s
5:	learn: 0.6213280	test: 0.6215503	best: 0.6215503 (5)	total: 2.81s	remaining: 7m 45s
6:	learn: 0.6257560	test: 0.6260429	best: 0.6260429 (6)	total: 3.3s	remaining: 7m 48s
7:	learn: 0.6289039	test: 0.6292071	best: 0.6292071 (7)	total: 3.79s	remaining: 7m 50s
8:	learn: 0.6346165	test: 0.6349570	best: 0.6349570 (8)	total: 4.32s	remaining: 7m 55s
9:	learn: 0.6373154	test: 0.6374683	best: 0.6374683 (9)	total: 4.79s	remaining: 7m 53s
10:	learn: 0.6391709	test: 0.6393966	best: 0.6393966 (10)	total: 5.32s	remaining: 7m 58s
11:	learn: 0

In [58]:
roc_auc_score(target_valid, cbc_model.predict(features_valid))

0.764369830071267

In [ ]:
pred = cbc_model.predict(df_test)

submission = pd.DataFrame({'sequence':df_test['sequence'], 'state':pred})
submission = submission.groupby('sequence').mean().reset_index()
submission['state'] = submission['state'].astype(int)

In [ ]:
submission.head()

,sequence,state
0,25968,0
1,25969,0
2,25970,0
3,25971,0
4,25972,0


In [ ]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12218 entries, 0 to 12217
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   sequence  12218 non-null  int64
 1   state     12218 non-null  int64
dtypes: int64(2)
memory usage: 191.0 KB


In [ ]:
submission.to_csv('/content/submission.csv', index=False)

In [ ]:
submission.shape

(12218, 2)